In [ ]:
# notebook 里自动绘制图像
%matplotlib inline 

import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
image_a = cv2.imread('output/20230712-175159/oginal29_child38.jpg')
image_b = cv2.imread('output/20230712-175159/oginal30_child31.jpg')
image_c = cv2.imread('output/20230712-175159/oginal35_child36.jpg')

In [ ]:
plt.figure(figsize= (20,20))

plt.subplot(3,3,1)
plt.imshow(image_a)

result = cv2.subtract(image_a, image_b) 
result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
plt.subplot(3,3,2)
plt.imshow(result)


result2 = cv2.absdiff(result, image_c) 
plt.subplot(3,3,3)
plt.imshow(result2)

In [ ]:
# 将图像A和B转换为灰度图像
gray_a = cv2.cvtColor(image_a, cv2.COLOR_BGR2GRAY)
gray_b = cv2.cvtColor(image_b, cv2.COLOR_BGR2GRAY)

# 对图像B进行阈值处理，生成掩膜
_, thresh_a = cv2.threshold(gray_a, 1, 255, cv2.THRESH_BINARY)
# 执行轮廓检测
contours_a, _ = cv2.findContours(thresh_a, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

areas = [cv2.contourArea(c) for c in contours_a]
max_idx = np.argmax(areas)
max_area_cnt = contours_a[max_idx]

# 获取轮廓的边界框
x, y, w, h = cv2.boundingRect(max_area_cnt)

print(x,y,h,w)

if w > 0 and h > 0:
    # 创建对应尺寸的空白图像
    result = np.zeros_like(image_a)
    # result[:, :, 3] = 0

    # 将轮廓绘制在空白图像上
    cv2.drawContours(result, [max_area_cnt], -1, (255, 255, 255), thickness=cv2.FILLED)
    extracted_part = cv2.bitwise_and(image_a, result)

    # 裁剪空白图像，仅保留轮廓区域
    extracted_part = extracted_part[y:y+h, x:x+w]
    showRes = cv2.cvtColor(extracted_part,cv2.COLOR_BGR2RGB)
    plt.imshow(showRes)

    # cv2.imwrite('./test001.png',showRes)
    
else:
    print("error")


In [ ]:
# 转换图像为RGB颜色空间
image_rgb = cv2.cvtColor(extracted_part, cv2.COLOR_BGR2RGB)

# 将黑色像素排除在直方图统计之外
mask = cv2.cvtColor(extracted_part, cv2.COLOR_BGR2GRAY)
mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)[1]
image_rgb[mask == 0] = (0, 0, 0)

# 计算非黑色像素的直方图
histogram = cv2.calcHist([image_rgb], [0, 1, 2], mask, [256, 256, 256], [0, 256, 0, 256, 0, 256])

# 找到像素数量最多的非黑色颜色索引
indices = np.unravel_index(np.argmax(histogram), histogram.shape)

# 获取像素数量最多的非黑色颜色
most_common_color = (indices[2], indices[1], indices[0])

# 显示结果
print("Most common non-black color:", most_common_color)

result = np.zeros((100, 100, 3), dtype=np.uint8)
result[:, :] = most_common_color

result = cv2.cvtColor(result,cv2.COLOR_BGR2RGB)

plt.imshow(result)


In [ ]:
img_filled = np.zeros_like(image_a)

cv2.drawContours(img_filled, [max_area_cnt], -1, (int(indices[2]), int(indices[1]), int(indices[0])),thickness=cv2.FILLED)
img_filled = cv2.cvtColor(img_filled,cv2.COLOR_BGR2RGB)

plt.figure(figsize= (10,10))
plt.imshow(img_filled)
